## Importing Stock Data with Yahoo Finance

- In this notebook you will find a Python script that provides a stock dataset that comes from Yahoo Finance. This dataset will be used for analysis with PowerBI.

### 1. Importing Librarys

In [3]:
import pandas as pd
import yfinance as yf
import psycopg2

In [ ]:
connection = psycopg2.connect(
    host="",
    database="",
    user="",
    port="",
    password="")


In [26]:
cursor = connection.cursor()

### 2. Setting some parameters:

In [11]:
stocks = ['F', 'AAPL', 'NVDA', 'TSLA', 'AMZN', 'MSFT', 'GOOGL', 'META', 'NFLX', 'AMD']
start_date = '2015-07-11'
end_date = '2025-07-11'

### 3. Making a Stock Dataset:

In [12]:
historical_data = []
fundamental_data = []
    
for symbol in stocks:

    #symbol = symbol + ".SA"
    ticker = yf.Ticker(symbol)

    # 🔹 Dados históricos
    hist = ticker.history(start=start_date, end=end_date)
    hist = hist.reset_index()
    hist['Symbol'] = symbol
    historical_data.append(hist)

    # 🔹 Dados fundamentalistas (P/L, P/VP, ROE etc.)
    info = ticker.info
    fundamentals = {
        'Symbol': symbol,
        'P/L (TTM)': info.get('trailingPE'),  
        'P/VP': info.get('priceToBook'),
        'ROE': info.get('returnOnEquity'),
        'Dividend Yield': info.get('dividendYield'),
    }
    fundamental_data.append(fundamentals)



### 4. Importing the Dataset into a "csv" file:

In [ ]:
# Exportar CSVs
df_prices = pd.concat(historical_data)
#df_prices.to_csv('dataset/stock-analysis.csv', index=False)
dataset_fundamentals = pd.DataFrame(fundamental_data)
#dataset_fundamentals.to_csv('dataset/stock-analysis-fundamentals.csv', index=False)
dataset_img = pd.read_csv('dataset/images.csv')

### 5. Inserting of the Dataset into PostgresSQL:

In [ ]:
for _, row in df_prices.iterrows():
    try:
        cursor.execute(
            """
            INSERT INTO StockData 
            (DateStock, SymbolStock, ClosePrice, HighPrice, OpenPrice, LowPrice, Volume) 
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """,
            (
                row['Date'].date() if hasattr(row['Date'], 'date') else row['Date'],  # Garante que será um DATE
                row['Symbol'],           
                round(row['Close'], 2),
                round(row['High'], 2),
                round(row['Open'], 2),
                round(row['Low'], 2),
                int(row['Volume'])       
            )
        )
        connection.commit()
    except Exception as e:
        print(f"Erro ao inserir o registro da data {row['Date']}: {e}")
        connection.rollback()


In [18]:
for _, row in dataset_fundamentals.iterrows():
    try:
        cursor.execute(
            """
            INSERT INTO FundamentalsStockData 
            (SymbolStock, DividendYield, PL, PVP, ROE) 
            VALUES (%s, %s, %s, %s, %s)
            """,
            (
                row['Symbol'], 
                round(row['Dividend Yield'], 2),
                round(row['P/L (TTM)'], 2),
                round(row['P/VP'], 2),
                round(row['ROE'], 2)
            )
        )
        connection.commit()
    except Exception as e:
        print(f"Erro ao inserir o registro do Ativo {row['SymbolStock']}: {e}")
        connection.rollback()


In [27]:
for _, row in dataset_img.iterrows():
    try:
        cursor.execute(
            """
            INSERT INTO stockimages 
            (SymbolStock, linkimage) 
            VALUES (%s, %s)
            """,
            (
                row['Symbol'], 
                row['Logo']
            )
        )
        connection.commit()
    except Exception as e:
        print(f"Erro ao inserir o registro da Imagem do Ativo {row['Symbol']}: {e}")
        connection.rollback()